# Analysis of datacube data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import plotPS as pps 
from figure import CustomFigure

Some information about the boxes and limiting frequencies:

In [ ]:
boxsize = 5120 #Mpc
ngrid = 256 #px
resolution = boxsize/ngrid #Mpc/px
k_nyquist = np.pi / resolution 
k_boxsize = 2*np.pi/boxsize

I will denote the power spectra outputed by gevolution with a superscript $gev$, by class with $cls$ and camb $cmb$. Thus, we have the following quantities:

* $P_\delta^{gev}(k)$ -> Matter power spectrum from gevolution
* $P_\delta^{cls}(k)$ -> Matter power spectrum from CLASS
* $P_\delta^{cmb}(k)$ -> Matter power spectrum from CAMB
* $P_\phi^{gev}(k)$ -> Potential power spectrum from gevolution
* $P_\phi(k)$ -> Potential power spectrum directly calculated from the datacube, using Pylians

We start off with plotting the matter power spectra generated by gevolution itself, $P_\delta^{gev}(k)$, for three different seeds and at three different redshifts, namely $z\in[0,1,10]$

In [ ]:
datapath = "/mn/stornext/d10/data/johanmkr/simulations/gevolution_first_runs/"

seeds = [0000, 1234, 1999]
pk_type = "delta" 
redshift = 0
add_objects = [pps.AddPowerSpectraComponents(datapath +f"seed{seeds[i]:04d}/") for i in range(len(seeds))]

def compare_seeds(ps_objects, pk_type, redshift):
    temp_fig = CustomFigure()
    colors = ["r", "g", "b"]
    settings = {
            "xscale": "log",
            "yscale": "log",
            "xlabel": r"k $[h/Mpc]$",
            "ylabel": r"P(k) $[h/Mpc]^3$",
            "title": f"Power spectrum of '{pk_type} at redshift z={redshift:.1f}, for seeds {seeds}",
            "xlim": (k_boxsize, k_nyquist),
            "ylim": (1e0, 1e5)
        }
    temp_fig.set_settings(settings)
    lines = []
    for i in range(len(ps_objects)):
        lines.extend(add_objects[i].add_gr_newton_gev(pk_type=pk_type, redshift=redshift, color=colors[i]))

    
    lines.append(add_objects[0].add_CAMB_spectrum(redshift, color="purple"))
    lines.append(add_objects[0].add_CLASS_spectrum(redshift, color="orange"))

    for line in lines:
        temp_fig.ax.add_line(line)

    leg1 = temp_fig.ax.legend(handles=lines[:6], labels=seeds, loc="upper right")
    leg2 = temp_fig.ax.legend(handles=lines[6:], labels=["CAMB", "CLASS"], loc="lower right")
    temp_fig.ax.add_artist(leg1)
    temp_fig.ax.add_artist(leg2)
    add_objects[0]._add_gr_newton_legend(temp_fig.ax)


compare_seeds(add_objects, "delta", redshift=0)
compare_seeds(add_objects, "delta", redshift=1)
compare_seeds(add_objects, "delta", redshift=10)


We can plot the same for the potential, but now 